In [6]:
!pip install pyspark

zsh:1: command not found: pip


In [1]:
from pyspark import SparkConf, SparkContext

# Create a Spark configuration object
conf = SparkConf().setAppName("App_Name")
# Set any additional configuration properties
conf.set("spark.executor.memory", "4g")
conf.set("spark.executor.cores","4")
conf.set("spark.driver.memory", "2g")

#create Spark Context
sc = SparkContext(conf=conf)

#Create an rdd and read data from a text file
rdd = sc.textFile("/Users/krishnaveni/desktop/textFile")
print("Initial partiton count:", rdd.getNumPartitions())

rddRepartitioned = rdd.repartition(4)
print("re-partiton count:", rdd.getNumPartitions())

#flatmap transformation () to split the words in the file by space
rdd_words = rdd.flatMap(lambda k: k.split(" "))

#map transformation is applied on the rdd to create a new radd of tuples where each word is mapped to value of 1 
rdd_map = rdd_words.map(lambda m : (m,1))

#filter transformation to filter the wrds starting with alphabet a
filtered_rdd = rdd_map.filter(lambda f:f[0].startswith("a"))

#reduce by transformation to compute count of each word
rdd_reduced_everyword = rdd_map.reduceByKey(lambda x,y: x+y)

#swap the word and count in the above rdd and then use the sort by to sort them based on the highest word count
rdd_sort = rdd_reduced_everyword.map(lambda s: (s[1],s[0])).sortByKey()

rdd_sort.foreach(print)

print("Count:", rdd_sort.count())


#total wordcount here we are using reduce action and the applying a lambda function to add the count from each tuple of count, word
#the result of the below is total count and first word
#print will just print the total word count
totalWordCount = rdd_sort.reduce(lambda a, b: (a[0] + b[0], a[1]))
print("dataReduce Record:", totalWordCount[0])

23/04/21 10:46:17 WARN Utils: Your hostname, KVs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.4.26 instead (on interface en0)
23/04/21 10:46:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/04/21 10:46:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/04/21 10:46:19 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Initial partiton count: 2
re-partiton count: 2


/Users/nikhilkatte/opt/anaconda3/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling
/Users/nikhilkatte/opt/anaconda3/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling
(1, 'am')                                                                       
(1, 'krishnaveni')
(1, 'I')
(1, 'Krishnaveni')(3, 'hi')

(3, 'hello')
(2, 'are')
(2, 'how')
(2, 'you')
(2, 'bannurkar')


Count: 10
dataReduce Record: 18
